<a href="https://colab.research.google.com/github/zeynep68/CIFAR100/blob/main/ResNet34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torchvision.datasets import CIFAR100 
from torchvision import transforms
from torch.utils.data import DataLoader, sampler
from torch.nn import Sequential, Conv2d, BatchNorm2d, ReLU, MaxPool2d, AvgPool2d, Softmax, Flatten

# Load and prepare data

In [96]:
BATCH_SIZE = 128
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

In [97]:
train = CIFAR100(root='./trainset', train=True, download=True, transform=transform)
test = CIFAR100(root='./testset', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [98]:
len(train), len(test)

(50000, 10000)

In [99]:
NUM_TRAIN = 49000
train_loader = DataLoader(train, batch_size=BATCH_SIZE, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN))) # 391
val_loader = DataLoader(train, batch_size=BATCH_SIZE, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))
test_loader = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False)

# ResNet


In [100]:
class ConvolutionalModule(nn.Module):
  def __init__(self, input_channel=64, output_channel=128):
    super(ConvolutionalModule, self).__init__()
    self.layer1 = Sequential(
                    Conv2d(in_channels=input_channel, out_channels=output_channel, kernel_size=3, stride=2, padding=1),
                    BatchNorm2d(output_channel), # über channels des outputs
                    ReLU()
                  )
    self.layer2 = Sequential(
                    Conv2d(in_channels=output_channel, out_channels=output_channel, kernel_size=3, stride=1, padding=1),
                    BatchNorm2d(output_channel), # über channels des outputs
                    ReLU()
                )    

    self.x_shortcut = Sequential(
                        Conv2d(in_channels=input_channel, out_channels=output_channel, kernel_size=1, stride=2, padding=0),
                        BatchNorm2d(output_channel),
                        ReLU()
                      )        

  def forward(self, x):
    return self.x_shortcut(x) + self.layer2(self.layer1(x))

In [101]:
class IdentityModule(nn.Module):
  def __init__(self, num_channel):
    super(IdentityModule, self).__init__()
    self.layer1 = Sequential(
                    Conv2d(in_channels=num_channel, out_channels=num_channel, kernel_size=3, stride=1, padding=1),
                    BatchNorm2d(num_channel), # über channels des outputs
                    ReLU()
                  )
    self.layer2 = Sequential(
                    Conv2d(in_channels=num_channel, out_channels=num_channel, kernel_size=3, stride=1, padding=1),
                    BatchNorm2d(num_channel),
                    ReLU()
                )          

  def forward(self, x):
    return x + self.layer2(self.layer1(x))

In [102]:
class ResNet34(nn.Module):
  def __init__(self):
    super(ResNet34, self).__init__()
    self.conv1 = Sequential(
                         Conv2d(in_channels=3, out_channels=64, kernel_size=7 ,stride=2, padding=0),
                         BatchNorm2d(64),
                         ReLU(),
                         MaxPool2d(kernel_size=3, stride=2)
                       )
    
    self.conv2 = Sequential(
                   IdentityModule(64),
                   IdentityModule(64),
                   IdentityModule(64)
                 )

    self.conv3 = Sequential(
                   ConvolutionalModule(),
                   IdentityModule(128),
                   IdentityModule(128),
                   IdentityModule(128)
                 )
    
    self.conv4 = Sequential(
                  ConvolutionalModule(128, 256),
                  IdentityModule(256),
                  IdentityModule(256),
                  IdentityModule(256),
                  IdentityModule(256),
                  IdentityModule(256)
                )
    
    self.conv5 = Sequential(
                   ConvolutionalModule(256, 512),
                   IdentityModule(512),
                   IdentityModule(512)
                 )

    self.output_layer = Sequential(
                          AvgPool2d(kernel_size=1),
                          Flatten(),
                          Softmax(dim=1)
                        )
    

  def forward(self, x):
    y = self.conv1(x)
    y = self.conv2(y)
    y = self.conv3(y)
    y = self.conv4(y)
    y = self.conv5(y)
    y = self.output_layer(y)
    return y

# Function to check accuracy

In [103]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

print('using device:', DEVICE)

using device: cuda


In [133]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print('Checking accuracy on validation set')
  else:
    print('Checking accuracy on test set') 
  model.eval() 

  num_correct = 0
  num_samples = 0

  with torch.no_grad():
    for images, labels in loader: # durchläuft 391 mal (mit jeweils 128 Bildern)
      images = images.to(device=DEVICE)
      labels = labels.to(device=DEVICE)
      scores = model(images)
      _, predictions = scores.max(1)
      num_correct += (predictions == labels).sum()
      num_samples += predictions.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

# Training

In [134]:
print_every = 100

In [141]:
def train(model, optimizer, epochs=30):

    model = model.to(device=DEVICE)
    for e in range(epochs):
        for t, (images, labels) in enumerate(train_loader): # durchläuft 391 mal (mit jeweils 128 Bildern)
            model.train() 
            images = images.to(device=DEVICE)
            labels = labels.to(device=DEVICE)

            scores = model(images)
            loss = F.cross_entropy(scores, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy(val_loader, model)
                print()

In [144]:
learning_rate = 1e-2
model = ResNet34()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [146]:
train(model, optimizer)

Iteration 0, loss = 6.0746
Checking accuracy on validation set
Got 147 / 1000 correct (14.70)



KeyboardInterrupt: ignored

# Evaluating